In [1]:
import numpy as np
from scipy.linalg import lu, lu_factor, lu_solve
from time import time

We'll start with a system of linear equations in matrix form ($Ax = b$).  As a concrete example,

$$
\left(
\begin{array}{llll}
4 & -2 & -3 &  1 \\
1 & 3 & 1 & 3 \\
1 & 2 & -1 & -2 \\
2 & 1 & -1 &-1
\end{array}
\right)\left(
\begin{array}{l}
x_1 \\
x_2 \\
x_3 \\
x_4
\end{array}
\right)=\left(
\begin{array}{l}
20 \\
14 \\
3 \\
9
\end{array}
\right)
$$

LU decomposition facturs a matrix $A$ into the product of two other matrices $A = LU$

$$
\left(
\begin{array}{llll}
4 & -2 & -3 &  1 \\
1 & 3 & 1 & 3 \\
1 & 2 & -1 & -2 \\
2 & 1 & -1 &-1
\end{array}
\right)
=
\left(
    \begin{array}{llll}
        1     &     0   &        0   &      0 \\
        0.25  &     1   &        0   &      0 \\
        0.25  &     0.71428571 & 1   &      0 \\
        0.5   &     0.57142857 & 0.33333333& 1
    \end{array}
\right) \left(
\begin{array}{llll}
4    &     -2.    &     -3     &      1          \\
0    &      3.5   &      1.75  &      2.75       \\
0    &      0     &     -1.5   &     -4.21428571 \\
0    &      0     &      0     &     -1.66666667
\end{array}
\right)
$$

Our equation is now,

$$
\left(
    \begin{array}{llll}
        1     &     0   &        0   &      0 \\
        0.25  &     1   &        0   &      0 \\
        0.25  &     0.71428571 & 1   &      0 \\
        0.5   &     0.57142857 & 0.33333333& 1
    \end{array}
\right) \underbrace{ \left(
\begin{array}{llll}
4    &     -2.    &     -3     &      1          \\
0    &      3.5   &      1.75  &      2.75       \\
0    &      0     &     -1.5   &     -4.21428571 \\
0    &      0     &      0     &     -1.66666667
\end{array}
\right)
\left(
\begin{array}{l}
x_1 \\
x_2 \\
x_3 \\
x_4
\end{array}
\right)}_\tilde{x} =\left(
\begin{array}{l}
20 \\
14 \\
3 \\
9
\end{array}
\right)
$$

After making that substitution, we have

$$
\left(
    \begin{array}{llll}
        1     &     0   &        0   &      0 \\
        0.25  &     1   &        0   &      0 \\
        0.25  &     0.71428571 & 1   &      0 \\
        0.5   &     0.57142857 & 0.33333333& 1
    \end{array}
\right)
\left(
\begin{array}{l}
\tilde{x_1} \\
\tilde{x_2} \\
\tilde{x_3} \\
\tilde{x_4}
\end{array}
\right) = 
\left(
\begin{array}{l}
20 \\
14 \\
3 \\
9
\end{array}
\right)
$$

Which we can solve for $\tilde{x}$ by forward substitution.

Finally, we can solve for our unknown $x$ using the dfinition of $\tilde{x}$ and using backward substitution.
$$
\left(
\begin{array}{llll}
4    &     -2.    &     -3     &      1          \\
0    &      3.5   &      1.75  &      2.75       \\
0    &      0     &     -1.5   &     -4.21428571 \\
0    &      0     &      0     &     -1.66666667
\end{array}
\right)
\left(
\begin{array}{l}
x_1 \\
x_2 \\
x_3 \\
x_4
\end{array}
\right) = 
\left(
\begin{array}{l}
\tilde{x_1} \\
\tilde{x_2} \\
\tilde{x_3} \\
\tilde{x_4}
\end{array}
\right) 
$$

In [2]:
#  Define a system of linear equations. A  is the coefficient matrix and b is the vector of knowns
#  We are using the same matrix as above
A = np.array( [ [4,-2,-3,1], [1,3,1,3], [1, 2, -1, -2], [2,1,-1,-1] ])
b = np.array([20, 14, 3, 9]); b.shape = (4, 1)

#  Do the matrix factorization.  In our case, the permutation matrix P is the identity
P, L, U = lu(A)

print('The L matrix is:')
print(L)
print()
print('The U matrix is:')
print(U)

#  Show the A = PLA
np.allclose(L @ U, A)

The L matrix is:
[[1.         0.         0.         0.        ]
 [0.25       1.         0.         0.        ]
 [0.25       0.71428571 1.         0.        ]
 [0.5        0.57142857 0.33333333 1.        ]]

The U matrix is:
[[ 4.         -2.         -3.          1.        ]
 [ 0.          3.5         1.75        2.75      ]
 [ 0.          0.         -1.5        -4.21428571]
 [ 0.          0.          0.         -1.66666667]]


True

We can solve the system of equations with LU decomposition via the lu_factor and lu_solve commands

In [5]:
#  Do the factorization
LU, p = lu_factor(A)

#  Solve the system
x1 = lu_solve((LU, p), b)
print(x1)

#  Does this give the same results as linalg.solve?
np.allclose(x1, np.linalg.solve(A, b))

[[ 5.0000000e+00]
 [ 1.0000000e+00]
 [-2.4319171e-15]
 [ 2.0000000e+00]]


True

If we have cases where the coefficient matrix $A$ is always the same, and we have differnt values of $b$, we can get a significant performance enhancement using LU decomposition over the normal solve command.

Below, we generate a lunear system with 500 unknowns and repeatedly solve it using solve and lu_solve to show the speed difference.

We will solve the system 10000 time each.

In [6]:
#  Set a constant seed so we can reproduce our results
np.random.seed(2)

#  Set number of equations
N = 500

#  Generate A matrix
A = np.random.normal( size = (N, N) )

#  make b vector
b = np.random.normal( size = (N, ))

###  Solve via linalg.solve

In [7]:
start = time()
for i in range(10000):
    x1 = np.linalg.solve(A, b)
print('Time = ', time() - start)

Time =  24.728508472442627


###  Solve via lu_solve

In [8]:
start = time()

lu, piv = lu_factor(A)
for i in range(10000):
    x2 = lu_solve((lu, piv), b)

print('Time = ', time() - start)

Time =  0.9258191585540771


In [27]:
#  let's make sure the answers from solve and lu_solve are the same.
np.allclose(x1, x2)

True